# Dog vs. Cat Detection
This notebook presents an entire fully working convolutional neural network to classify dog vs. cat images. I am having trouble running it on the kaggle notebook server because I keep getting an "out of memory"/other resources error. I got it to work on my own machine with smaller number of training images.

## Import Training Data
This section will prepare and import the training data images

In [ ]:
TRAIN_DIR = '../input/train' #training directory
TEST_DIR = '../input/test' #testing directory


Pre-define a function that will take the labels from the image name (thanks to other kernels on Kaggle).

In [ ]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0] #[cat, not dog]
    elif word_label == 'dog': return [0,1] #[not cat, dog]

Define a function that will retrieve the training data:

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm 
IMG_SIZE=64 #Define short image size

def create_train_data():
    X = [] #training images
    Y = [] #training labels
    for img in tqdm(os.listdir(TRAIN_DIR)[5000:20000]):
        label = label_img(img)
        path=os.path.join(TRAIN_DIR, img)
        im=cv2.imread(path)
        sh=im.shape
        if sh[0] < sh[1]:  # Find the shortest dimension
            im1=im[0:sh[0],0:sh[0]]
            im2=im[0:sh[0],(sh[1]-sh[0]):sh[1]]
            #rint(""
        else:
            im1=im[0:sh[1],0:sh[0]]
            im2=im[(sh[0]-sh[1]):sh[0],0:sh[1]]
        #Resize images to make them smaller
        im1 = cv2.resize(im1, (IMG_SIZE,IMG_SIZE))
        im2 = cv2.resize(im2, (IMG_SIZE,IMG_SIZE))
        #Append the two images together
        imn=np.append(im1,im2,axis=0)
        X.append(np.array(imn))
        Y.append(np.array(label))
    return(X,Y)
    

Run the new function to create the training data.

In [ ]:
X,Y = create_train_data()

In [ ]:
#Convert the training data to an appropriate filetype
Y=np.asarray(Y)
Y=Y.astype(np.float32) #Now Y is fixed to a datatype

X=np.asarray(X)
#type(X[1,1,1,1])
X=(X/256).astype(np.float32)
print("Done converting the files to float64")

Try some of the images to make sure what they look like:

In [ ]:
import matplotlib.pyplot as plt
import random
from tflearn.data_utils import shuffle

#index=random.randint(0,25000-1)
#plt.imshow(X[index])
#Y[index]
#X[index].shape

X, Y = shuffle(X, Y)

Xtrain=X[0:13000]
Ytrain=Y[0:13000]
Xtest=X[13000:15000]
Ytest=Y[13000:15000]


## Define the Neural Network
This next section will define the neural network for the detector

In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

#Pre-process the images
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)


# Convolutional network building
network = input_data(shape=[None, IMG_SIZE*2, IMG_SIZE, 3], #Define input layer
                     data_preprocessing=img_prep, #set data_preprocessing
                     data_augmentation=img_aug)

network = conv_2d(network, 32,3, activation='relu') #convolve data once
network = max_pool_2d(network,2) #down sample (reduce data)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network,2) #down sample (reduce data)
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 2, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

Run the training iterations repeatedly

In [ ]:
#import scipy

#Define the model
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='dog-cat.tfl.ckpt')
model.fit(Xtrain, Ytrain, n_epoch=10, 
          shuffle=True, 
          validation_set=(Xtest, Ytest),
          show_metric=True, 
          batch_size=96, 
          run_id='catvdog')

## Preparation
Now the model is trained. Time to prepare the training data and see what can be done.

In [ ]:
def prepare_testing_data():
    Xtest = [] #training images
    filenames = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path=os.path.join(TEST_DIR, img)
        im=cv2.imread(path)
        sh=im.shape
        if sh[0] < sh[1]:  # Find the shortest dimension
            im1=im[0:sh[0],0:sh[0]]
            im2=im[0:sh[0],(sh[1]-sh[0]):sh[1]]
            #rint(""
        else:
            im1=im[0:sh[1],0:sh[0]]
            im2=im[(sh[0]-sh[1]):sh[0],0:sh[1]]
        #Resize images to make them smaller
        im1 = cv2.resize(im1, (IMG_SIZE,IMG_SIZE))
        im2 = cv2.resize(im2, (IMG_SIZE,IMG_SIZE))
        #Append the two images together
        imn=np.append(im1,im2,axis=0)
        Xtest.append(np.array(imn))
        filenames.append(img)
    return(Xtest,filenames)
    

In [ ]:
Xcomp1,fn = prepare_testing_data() #load a portion of the testing data

In [ ]:
Xcomp1=np.asarray(Xcomp1)
Xcomp1=(Xcomp1/256).astype(np.float64)
print("Done")

Now run the model on the testing data

In [ ]:
ind=random.randint(0,5000)

path=os.path.join(TEST_DIR, fn[ind])
im=cv2.imread(path)
plt.imshow(im)
plt.show()

plt.imshow(Xcomp1[ind])
plt.show()

k=model.predict(np.reshape(Xcomp1[ind],(1,100,50,3)))

print(fn[ind])
print("Cat         Dog")
print("%2.4f     %2.4f" % (k[0][0], k[0][1]) )

## Create the submission CSV file
The next code section will create a submission .CSV file. 

In [ ]:
import csv
fn

index_array=np.empty([12500,1]).astype(np.uint16)
pred_array=np.empty([12500,1]).astype(np.float64)


index=int(i[:-4])


p=0
for i in tqdm(fn):
    index=int(i[:-4])
    
    pred=model.predict(np.reshape(Xcomp1[p],(1,100,50,3)))
    
    index_array[index-1]=index
    pred_array[index-1]=pred[0][1]
    p=p+1

Now write the results to the submission file.

In [ ]:
output=np.column_stack((index_array,pred_array))
np.savetxt('submission.csv',
           output,
           fmt='%d, %1.4f',
           delimiter=',',
           header='id,label')